In [ ]:
# === colab configuration ===
# p.s. when training on colab, weights are saved on Drive (directory DSCAS/weights).
# p.p.s. skip this cell if running demo file locally!

! pip install torch-summary
! pip install ray
! pip install tensorboardX==2.5.1
! pip install tensorboard==2.11.2
from google.colab import drive, files
import sys

# setting paths
repository_path = '/content/deep-seasonal-color-analysis-system/'
dataset_path = repository_path + 'headsegmentation_dataset_ccncsa/'
dataset_path_drive = '/content/drive/MyDrive/DSCAS/headsegmentation_dataset_ccncsa/'
weights_path = repository_path + 'models/weights/'
weights_path_drive = '/content/drive/MyDrive/DSCAS/weights/'
checkpoint_path = repository_path + 'models/preprocessing/'
checkpoint_archive = '/content/' + checkpoint_path.split('/')[-2] + '.zip'

sys.path.insert(0, repository_path)

# cloning project repository and downloading dataset
drive.mount('/content/drive')
! test ! -d $repository_path && git clone https://github.com/mrcmich/deep-seasonal-color-analysis-system.git
! test ! -d $dataset_path && cp -R $dataset_path_drive $dataset_path
%cd $repository_path

# setting branch and pulling updates
branch = 'main'
! git checkout $branch
! git pull origin $branch

executing_on_colab = True

In [2]:
import os
import torch
from torch import nn, optim
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from models import dataset, training_and_testing
from models.local.FastSCNN.models import fast_scnn
from metrics_and_losses import metrics
from utils import segmentation_labels, custom_transforms, model_names
from models import config
from slurm_scripts import slurm_config
from ray import tune
from ray.tune import CLIReporter
from functools import partial

try:
  executing_on_colab
except NameError:
  executing_on_colab = False
  weights_path = config.WEIGHTS_PATH
  dataset_path = config.DATASET_PATH
  checkpoint_path = config.PREPROCESSING_PATH

In [6]:
# === defining transforms ===

diameter = 7
sigma_color = 50
sigma_space = 100
horizontal_flip = custom_transforms.PartiallyDeterministicHorizontalFlip(p=0.5)
center_crop = custom_transforms.PartiallyDeterministicCenterCrop(p=0.5)

image_transform = T.Compose([
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

target_transform = T.Compose([T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

# fetching dataset
n_classes = len(segmentation_labels.labels)
img_paths, label_paths = dataset.get_paths(dataset_path, file_name=config.DATASET_INDEX_NAME)
X_train, _, Y_train, _ = train_test_split(img_paths, label_paths, test_size=0.20, random_state=99, shuffle=True)
train_dataset = dataset.CcncsaDataset(X_train, Y_train, image_transform, target_transform)

# setting up hyperparameters for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epochs = 10
batch_size = 32
loss_fn = nn.CrossEntropyLoss()
score_fn = metrics.batch_mIoU
learning_rate = 0.01
class_weights = torch.tensor(config.CLASS_WEIGHTS, device=device)

In [7]:
# model parameters
model_name = "fastscnn"
model = fast_scnn.FastSCNN(n_classes)
optimizer = optim.Adam

In [9]:
# Ray Tune parameters
cpus_per_trial = 0
gpus_per_trial = torch.cuda.device_count()
num_samples = 1  # Number of times each combination is sampled (n_epochs are done per sample)
metric = "val_loss"
metrics_columns = ["train_loss", "train_score", "val_loss", "val_score", "training_iteration"]
local_dir = checkpoint_path + model_names.MODEL_NAMES[model_name]
max_report_frequency = 600
reporter = CLIReporter(
    metric_columns=metrics_columns, max_report_frequency=max_report_frequency)

In [10]:
# === training without augmentation ===

checkpoint_dir = local_dir if executing_on_colab else (os.path.abspath('./' + local_dir) + '/')
cfg = {
    "lr": learning_rate,
    "lr_scheduler": "none",
    "batch_size": batch_size,
    "transform": "none",
    "from_checkpoint": False,
    "checkpoint_dir": checkpoint_dir
    }

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_2cb75_00000,2023-02-03_01-26-46,True,,8ef2415d89094266b7e14dbf30798615,0,lenovo-notebook,10,127.0.0.1,12600,2684.44,260.923,2684.44,1675384006,0,,0.13448,0.603462,10,2cb75_00000,0.131481,0.618394,0.00500274


2023-02-03 01:26:47,072	INFO tune.py:762 -- Total run time: 2692.60 seconds (2692.33 seconds for the tuning loop).


== Status ==
Current time: 2023-02-03 01:26:46 (running for 00:44:52.34)
Memory usage on this node: 6.2/7.4 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/12 CPUs, 0/1 GPUs, 0.0/1.34 GiB heap, 0.0/0.67 GiB objects
Result logdir: c:\Data\GitHub\deep-seasonal-color-analysis-system\models\preprocessing\FastSCNN\train_model_2023-02-03_00-41-54
Number of trials: 1/1 (1 TERMINATED)
+-------------------------+------------+-----------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status     | loc             |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+------------+-----------------+--------------+---------------+------------+-------------+----------------------|
| train_model_2cb75_00000 | TERMINATED | 127.0.0.1:12600 |      0.13448 |      0.603462 |   0.131481 |    0.618394 |                   10 |
+-------------------------+------------+-------

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab is True: files.download(checkpoint_archive)

In [11]:
# === training with augmentation: ColorJitter ===

image_transform_with_augmentation = T.Compose([
    T.ColorJitter(brightness=0.25, contrast=0.25), 
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

train_dataset_with_augmentation = dataset.CcncsaDataset(
    X_train, Y_train, image_transform_with_augmentation, target_transform)
    
model = fast_scnn.FastSCNN(n_classes)

cfg["transform"] = "color_jitter"

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset_with_augmentation, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_71c2f_00000,2023-02-03_02-14-55,True,,a255cb8b5e0d4d2b83b459b111b1b71d,0,lenovo-notebook,10,127.0.0.1,3468,2881.53,286.039,2881.53,1675386895,0,,0.124235,0.608309,10,71c2f_00000,0.146889,0.606491,0.01946


2023-02-03 02:14:55,941	INFO tune.py:762 -- Total run time: 2888.65 seconds (2888.39 seconds for the tuning loop).


== Status ==
Current time: 2023-02-03 02:14:55 (running for 00:48:08.40)
Memory usage on this node: 6.1/7.4 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/12 CPUs, 0/1 GPUs, 0.0/1.34 GiB heap, 0.0/0.67 GiB objects
Result logdir: c:\Data\GitHub\deep-seasonal-color-analysis-system\models\preprocessing\FastSCNN\train_model_2023-02-03_01-26-47
Number of trials: 1/1 (1 TERMINATED)
+-------------------------+------------+----------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status     | loc            |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+------------+----------------+--------------+---------------+------------+-------------+----------------------|
| train_model_71c2f_00000 | TERMINATED | 127.0.0.1:3468 |     0.124235 |      0.608309 |   0.146889 |    0.606491 |                   10 |
+-------------------------+------------+-----------

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab is True: files.download(checkpoint_archive)

In [12]:
# === training with augmentation: horizontal_flip ===

image_transform_with_augmentation = T.Compose([
    horizontal_flip,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

target_transform_with_augmentation = T.Compose([
    horizontal_flip,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

train_dataset_with_augmentation = dataset.CcncsaDataset(
    X_train, Y_train, image_transform_with_augmentation, target_transform_with_augmentation)
    
model = fast_scnn.FastSCNN(n_classes)

cfg["transform"] = "horizontal_flip"

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset_with_augmentation, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_2b9f4_00000,2023-02-03_03-00-57,True,,f4e057ca9f7e4fd5b42847a764b9ab8c,0,lenovo-notebook,10,127.0.0.1,12456,2753.36,275.702,2753.36,1675389657,0,,0.142049,0.589971,10,2b9f4_00000,0.149761,0.600934,0.00784945


2023-02-03 03:00:57,561	INFO tune.py:762 -- Total run time: 2761.47 seconds (2761.19 seconds for the tuning loop).


== Status ==
Current time: 2023-02-03 03:00:57 (running for 00:46:01.21)
Memory usage on this node: 6.2/7.4 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/12 CPUs, 0/1 GPUs, 0.0/1.34 GiB heap, 0.0/0.67 GiB objects
Result logdir: c:\Data\GitHub\deep-seasonal-color-analysis-system\models\preprocessing\FastSCNN\train_model_2023-02-03_02-14-56
Number of trials: 1/1 (1 TERMINATED)
+-------------------------+------------+-----------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status     | loc             |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+------------+-----------------+--------------+---------------+------------+-------------+----------------------|
| train_model_2b9f4_00000 | TERMINATED | 127.0.0.1:12456 |     0.142049 |      0.589971 |   0.149761 |    0.600934 |                   10 |
+-------------------------+------------+-------

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab is True: files.download(checkpoint_archive)

In [13]:
# === training with vaugmentation: center_crop ===

image_transform_with_augmentation = T.Compose([
    center_crop,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

target_transform_with_augmentation = T.Compose([
    center_crop,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

train_dataset_with_augmentation = dataset.CcncsaDataset(
    X_train, Y_train, image_transform_with_augmentation, target_transform_with_augmentation)

model = fast_scnn.FastSCNN(n_classes)

cfg["transform"] = "center_crop"

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset_with_augmentation, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_99b0e_00000,2023-02-03_03-45-26,True,,04e97dcbfe7f4eeaa0ea3d64b5d4514b,0,lenovo-notebook,10,127.0.0.1,18976,2661.2,265.631,2661.2,1675392326,0,,0.129888,0.615737,10,99b0e_00000,0.142262,0.617483,0.015414


2023-02-03 03:45:26,981	INFO tune.py:762 -- Total run time: 2669.25 seconds (2668.97 seconds for the tuning loop).


== Status ==
Current time: 2023-02-03 03:45:26 (running for 00:44:28.98)
Memory usage on this node: 6.1/7.4 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/12 CPUs, 0/1 GPUs, 0.0/1.34 GiB heap, 0.0/0.67 GiB objects
Result logdir: c:\Data\GitHub\deep-seasonal-color-analysis-system\models\preprocessing\FastSCNN\train_model_2023-02-03_03-00-57
Number of trials: 1/1 (1 TERMINATED)
+-------------------------+------------+-----------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status     | loc             |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+------------+-----------------+--------------+---------------+------------+-------------+----------------------|
| train_model_99b0e_00000 | TERMINATED | 127.0.0.1:18976 |     0.129888 |      0.615737 |   0.142262 |    0.617483 |                   10 |
+-------------------------+------------+-------

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab is True: files.download(checkpoint_archive)